<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/ResNet50_ImageNet_adabound_crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GravCont: EfficientNet_b4_ImageNet**
ValidationとTestに分けて解析

In [1]:
from __future__ import print_function, division
!pip install torch_optimizer

import torch
import torch.nn as nn
import torch_optimizer as optim
import torch.utils.data as data
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import shutil

#Advanced Pytorchから
import glob
import os.path as osp
import random
import json
from PIL import Image
from tqdm import tqdm
%matplotlib inline

import pandas as pd
import csv

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set random seem for reproducibility
manualSeed = 1234
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)

torch.torch.backends.cudnn.benchmark = True
torch.torch.backends.cudnn.enabled = True

!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet 

import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


'''
grav: 甲状腺眼症
cont: コントロール
黒の空白を挿入することにより225px*225pxの画像を生成、EfficientNetを用いて転移学習
－－－－－－－－－－－－－－
データの構造
gravcont.zip ------grav
               |---cont
'''                                     

#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


     |████████████████████████████████| 51kB 3.8MB/s 
Random Seed:  1234
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=c6a12bc4e0e6ee3400902d115579c5705dbe065e7b953e2de5af8621ef6fd92f
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
Mounted at /content/drive


#**モジュール群**

In [2]:
def pre_process(data_dir):
    # 入力画像の前処理をするクラス
    # 訓練時と推論時で処理が異なる

    """
        画像の前処理クラス。訓練時、検証時で異なる動作をする。
        画像のサイズをリサイズし、色を標準化する。
        訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


        Attributes
        ----------
        resize : int
            リサイズ先の画像の大きさ。
        mean : (R, G, B)
            各色チャネルの平均値。
        std : (R, G, B)
            各色チャネルの標準偏差。
    """

    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.75,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = data_dir
    n_samples = len(data_dir)

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=20,
                                                shuffle=True, num_workers=4)
                  for x in ['train', 'val']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes


    print(class_names)
    k=0
    for i in class_names:
        print(class_names[k]+"_train:"+str(len(os.listdir(path=data_dir + '/train/'+class_names[k]))))
        k+=1
    k=0
    for i in class_names:
        print(class_names[k]+"_val:"+str(len(os.listdir(path=data_dir + '/val/'+class_names[k]))))
        k+=1

    print("training data set_total："+ str(len(image_datasets['train'])))
    print("validating data set_total："+str(len(image_datasets['val'])))
    
    return image_datasets, dataloaders, dataset_sizes, class_names, device


#少数の画像を可視化
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


def getBatch(dataloaders):    
    # Get a batch of training data
    inputs, classes = next(iter(dataloaders['train']))

    # Make a grid from batch
    out = torchvision.utils.make_grid(inputs)

    #imshow(out, title=[class_names[x] for x in classes])
    return(inputs, classes)

#Defining early stopping class
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


#Train models
def train_model(model, criterion, optimizer, patience, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # to track the training loss as the model trains
    train_loss = []
    # to track the validation loss as the model trains
    valid_loss = []


    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase] 
            
            # record train_loss and valid_loss
            if phase == 'train':
                train_loss.append(epoch_loss)
            if phase == 'val':
                valid_loss.append(epoch_loss)
            #print(train_loss)
            #print(valid_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
      
      # early_stopping needs the validation loss to check if it has decresed, 
      # and if it has, it will make a checkpoint of the current model
        if phase == 'val':    
            early_stopping(epoch_loss, model)
                
            if early_stopping.early_stop:
                print("Early stopping")
                break
        print()


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss, valid_loss


#Visualize model
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


def training(model_ft, criterion, optimizer_ft,  patience=15, num_epochs=50):
    model_ft, train_loss, valid_loss = train_model(model_ft, criterion, optimizer_ft,  patience=patience, num_epochs=num_epochs)


#対象のパスからラベルを抜き出して表示
def getlabel(image_path):
      image_name = os.path.basename(image_path)
      label = os.path.basename(os.path.dirname(image_path))
      return(image_name, label)

'''
#変形後の画像を表示
def image_transform(image_path):

    image=Image.open(image_path)

    
    #変形した画像を表示する
    transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224)])
    image_transformed = transform(image)
    plt.imshow(np.array(image_transformed))
'''

#評価のための画像下処理
def image_transform(image_path):    
    image=Image.open(image_path)
    transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    image_tensor = transform(image)

    #バッチサイズの次元を先頭に追加した4Dテンソルに変換
    image_tensor.unsqueeze_(0)
    #print(image_tensor.size())  # torch.Size([1, 3, 224, 224])
    image_tensor = image_tensor.to(device) #model_ftをGPUに載せる

    return(image_tensor)

#モデルにした処理した画像を投入して予測結果を表示
def image_eval(image_tensor, label):
    output = model_ft(image_tensor)
    #print(output.size())  # torch.Size([1, 1000])
    #print(output)

    #model_pred:クラス名前、prob:確率、pred:クラス番号
    prob, pred = torch.topk(nn.Softmax(dim=1)(output), 1)
    model_pred = class_names[pred]
    
    #甲状腺眼症のprobabilityを計算（classが0なら1から減算、classが1ならそのまま）
    prob = abs(1-float(prob)-float(pred))
 
    return model_pred, prob, pred

    """
    #probalilityを計算する
    pred_prob = torch.topk(nn.Softmax(dim=1)(output), 1)[0]
    pred_class = torch.topk(nn.Softmax(dim=1)(output), 1)[1]
    if pred_class == 1:
        pred_prob = pred_prob
    elif pred_class == 0:
        pred_prob = 1- pred_prob
    return(model_pred, pred_prob)  #class_nameの番号で出力される
    """

def showImage(image_path):
    #画像のインポート
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    #画像のリサイズ
    height = img.shape[0]
    width = img.shape[1]
    resized_img = cv2.resize(img, (int(width*300/height), 300))
    cv2_imshow(resized_img)

def calculateAccuracy (TP, TN, FP, FN):
    accuracy = (TP + TN)/ (TP + TN + FP + FN)
    precision  = TP/(FP + TP)
    recall = TP/(TP + FN)
    specificity = TN/(FP + TN)
    f_value = (2*recall*precision)/(recall+precision)
    return(accuracy, precision, recall, specificity, f_value)

"""
・True positive (TN)
・False positive (FP)
・True negative (TN)
・False negative (FN)
Accuracy = (TP + TN)/ (TP + TN + FP + FN)
Precision = TP/(FP + TP) ※positive predictive value
Recall = TP/(TP + FN)　※sensitivity
Specificity = TN/(FP + TN)
F_value = (2RecallPrecision)/(Recall+Precision)
"""

def evaluation(model_ft, testset_dir):
    #評価モードにする
    model_ft.eval()

    #testデータセット内のファイル名を取得
    image_path = glob.glob(testset_dir + "/*/*")
    #random.shuffle(image_path)  #表示順をランダムにする
    print('number of images: ' +str(len(image_path)))


    TP, FP, TN, FN, TP, FP, TN, FN = [0,0,0,0,0,0,0,0]
    image_name_list = []
    label_list = []
    model_pred_list = []
    hum_pred_list = []

    model_pred_class = []
    model_pred_prob = []

    for i in image_path:
          image_name, label = getlabel(i)  #画像の名前とラベルを取得
          image_tensor = image_transform(i)  #予測のための画像下処理
          model_pred, prob, pred = image_eval(image_tensor, label)  #予測結果を出力   
          #print('Image: '+ image_name)
          #print('Label: '+ label)
          #print('Pred: '+ model_pred)
          #showImage(i)  #画像を表示
          #print() #空白行を入れる
          time.sleep(0.1)

          image_name_list.append(image_name)
          label_list.append(label)
          model_pred_list.append(model_pred)

          model_pred_class.append(int(pred))
          model_pred_prob.append(float(prob))

          if label == class_names[0]:
              if model_pred == class_names[0]:
                  TN += 1
              else:
                  FP += 1
          elif label == class_names[1]:
              if model_pred == class_names[1]:
                  TP += 1
              else:
                  FN += 1     

    print(TP, FN, TN, FP)

    #Accuracyを計算
    accuracy, precision, recall, specificity, f_value = calculateAccuracy (TP, TN, FP, FN)
    print('Accuracy: ' + str(accuracy))
    print('Precision (positive predictive value): ' + str(precision))
    print('Recall (sensitivity): ' + str(recall))
    print('Specificity: ' + str(specificity))
    print('F_value: ' + str(f_value))

    #print(model_pred_class)
    #print(model_pred_prob)

    return TP,TN,FP,FN, accuracy, precision, recall, specificity, f_value, label_list, model_pred_prob


def make_csv(roc_label_list):
    #csvのdata tableを作成
    pd.set_option('display.max_rows', 800)  # 省略なしで表示
    #columns1 = ["EfficientNet_32", "EfficientNet_64", "EfficientNet_128", "EfficientNet_256", "EfficientNet_512", "EfficientNet_558"]
    roc_label_list.extend(["avg", "std"])
    index1 = ["TP","TN","FP","FN","Accuracy","Positive predictive value","sensitity","specificity","F-value","roc_auc"]
    df = pd.DataFrame(index=index1, columns=roc_label_list)
    return df

def write_csv(df, col, TP, TN, FP, FN, accuracy, precision, recall, specificity, f_value,roc_auc):
    df.iloc[0:10, col] = TP, TN, FP, FN, accuracy, precision, recall, specificity, f_value,roc_auc 
    #print(df)

    # CSVとして出力
    #df2.to_csv("/content/drive/My Drive/Grav_bootcamp/Posttrain_model_eval_result.csv",encoding="shift_jis")
    return df

def Draw_roc_curve(label_list_list, model_pred_prob_list, sample_num_list, num_curves):

#グラフの外形を作成
    fig = plt.figure(figsize=(8.0, 6.0))
    lw = 2
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    ycolor = ["r", "g", "b", "c", "m", "y", "k", "w"]      # 各プロットの色
    plt.legend(loc="lower right")

    k=0
    for j in range(num_curves):
        y_score = []
        y_true = []

        for i in label_list_list[k]:
            if i == 'cont':
                  y_true.append(0)
            elif i == 'grav':
                  y_true.append(1)
            
        #それぞれの画像における陽性の確率についてリストを作成
        y_score = model_pred_prob_list[k]

        fpr, tpr,thred = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, color=ycolor[k],lw=lw, label= str(roc_label_list[k])+':ROC curve (area = %0.2f)' % roc_auc)
            
        k+=1

    plt.legend(loc="lower right")
    plt.show()
    return fig

def calculate_auc(label_list, model_pred_prob):
    y_true, y_score = [], []
    for i in label_list:
        if i == 'cont':
              y_true.append(0)
        elif i == 'grav':
              y_true.append(1)
            
    #それぞれの画像における陽性の確率についてリストを作成
    y_score = model_pred_prob
    fpr, tpr,thred = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    print("roc_auc: " +str(roc_auc))
    return(roc_auc, y_true, y_score)

def calcurate_ave_std(df, fold):
    for i in range(5):
        df.iloc[i,fold] = df[i,0:5].mean 

def convnet():
    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)

    #GPU使用
    model_ft = model_ft.to(device)

    #損失関数を定義
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    #https://blog.knjcode.com/adabound-memo/
    #https://pypi.org/project/torch-optimizer/
    optimizer_ft = optim.AdaBound(
        model_ft.parameters(),
        lr= 1e-3,
        betas= (0.9, 0.999),
        final_lr = 0.1,
        gamma=1e-3,
        eps= 1e-8,
        weight_decay=0,
        amsbound=False,
    )
    return (model_ft, criterion, optimizer_ft)

#**まとめて解析**

In [ ]:
# 出力名を記入
out_name = "ResNet50_ImageNet_128"

#create data_dir_list
data_dir = '/content/drive/My Drive/crossvalidation/128'
fold = len(os.listdir(data_dir))
print(str(fold)+'-fold cross validation')


data_dir_list = [0]*fold

for i in range(fold):
    data_dir_list[i] = data_dir + '/' + str(i)
    print(data_dir_list[i])

#create roc_label_list
roc_label_list = [0]*fold
roc_label_list = list(range(fold))
print(roc_label_list)



df = make_csv(roc_label_list)

label_list_list, model_pred_prob_list, Y_TRUE, Y_SCORE = [],[],[],[]

print(data_dir_list)
print(roc_label_list)

for i, t in enumerate(zip(data_dir_list, roc_label_list)):

    image_datasets, dataloaders, dataset_sizes, class_names, device = pre_process(t[0]) #path
    inputs, classes = getBatch(dataloaders)
    model_ft, criterion, optimizer_ft = convnet()
    training(model_ft, criterion, optimizer_ft,  patience=15, num_epochs=50)  
    TP,TN,FP,FN, accuracy, precision, recall, specificity, f_value, label_list, model_pred_prob = evaluation(model_ft, '/content/drive/My Drive/Grav_bootcamp/Posttrain_250px')
    roc_auc, y_true, y_score = calculate_auc(label_list, model_pred_prob)
    Y_TRUE.append(y_true)
    Y_SCORE.append(y_score)
    df = write_csv(df, i,TP,TN,FP,FN, accuracy, precision, recall, specificity, f_value, roc_auc) #numberをcsvの行として指定

    label_list_list.append(label_list)
    model_pred_prob_list.append(model_pred_prob)
    print("")
    print("")

#Draw ROC curve
fig = Draw_roc_curve(label_list_list, model_pred_prob_list, roc_label_list, len(label_list_list))

pd.set_option('display.max_columns', 100)
#それぞれの項目の平均を計算しcsvに追記する
df.iloc[0:4,fold], df.iloc[9,fold]   = df.mean(axis=1)[0:4], df.mean(axis=1)[9] 
df.iloc[0:10,fold+1] = df.std(axis=1)[0:10]
TP,TN,FP,FN = df.mean(axis=1)[0:4]
df.iloc[4:9,fold] = calculateAccuracy (TP, TN, FP, FN)
print(df)

# 出力名を記入(Network + number of images)
out_name = "ResNet50_ImageNet_256"

# CSVとして出力
df.to_csv("/content/drive/My Drive/Grav_bootcamp/crossvaridation_" + out_name + ".csv",encoding="shift_jis")

#ROC_curveを保存
fig.savefig("/content/drive/My Drive/Grav_bootcamp/crossvaridation_" + out_name +".png")

#Save ROC data
with open("/content/drive/My Drive/Grav_bootcamp/ROCdata_"+out_name+".csv", 'w') as f:
    writer = csv.writer(f)
    for i, t in enumerate(zip(Y_TRUE, Y_SCORE)):
        writer.writerow([t[0],t[1]])




5-fold cross validation
/content/drive/My Drive/crossvalidation/128/0
/content/drive/My Drive/crossvalidation/128/1
/content/drive/My Drive/crossvalidation/128/2
/content/drive/My Drive/crossvalidation/128/3
/content/drive/My Drive/crossvalidation/128/4
[0, 1, 2, 3, 4]
['/content/drive/My Drive/crossvalidation/128/0', '/content/drive/My Drive/crossvalidation/128/1', '/content/drive/My Drive/crossvalidation/128/2', '/content/drive/My Drive/crossvalidation/128/3', '/content/drive/My Drive/crossvalidation/128/4']
[0, 1, 2, 3, 4, 'avg', 'std']
['cont', 'grav']
cont_train:51
grav_train:51
cont_val:13
grav_val:13
training data set_total：102
validating data set_total：26


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



Epoch 0/49
----------


/usr/local/lib/python3.6/dist-packages/torch_optimizer/adabound.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


train Loss: 1.1259 Acc: 0.5098
val Loss: 262.2727 Acc: 0.5000
Validation loss decreased (inf --> 262.272696).  Saving model ...

Epoch 1/49
----------
train Loss: 0.7963 Acc: 0.5098
val Loss: 43.7399 Acc: 0.5000
Validation loss decreased (262.272696 --> 43.739904).  Saving model ...

Epoch 2/49
----------
train Loss: 0.7353 Acc: 0.5490
val Loss: 6.9226 Acc: 0.5000
Validation loss decreased (43.739904 --> 6.922581).  Saving model ...

Epoch 3/49
----------
train Loss: 0.7356 Acc: 0.5490
val Loss: 3.0005 Acc: 0.5000
Validation loss decreased (6.922581 --> 3.000458).  Saving model ...

Epoch 4/49
----------
train Loss: 0.7498 Acc: 0.5196
val Loss: 1.6048 Acc: 0.5000
Validation loss decreased (3.000458 --> 1.604772).  Saving model ...

Epoch 5/49
----------
train Loss: 0.5453 Acc: 0.7353
val Loss: 0.5965 Acc: 0.6923
Validation loss decreased (1.604772 --> 0.596479).  Saving model ...

Epoch 6/49
----------
train Loss: 0.4452 Acc: 0.7745
val Loss: 1.3166 Acc: 0.6923
EarlyStopping counter: 1

#**ネットワークの保存と読み込み**

In [ ]:
#ネットワークの保存
PATH = '/content/drive/My Drive/Grav_bootcamp/GravCont_EfficientNet-b4_ImageNet_seed'+str(manualSeed)+'.pth'
torch.save(model_ft.state_dict(), PATH)

In [ ]:
#ネットワークの読み込み
PATH = '/content/drive/My Drive/Grav_bootcamp/GravCont_EfficientNet-b4_ImageNet_seed'+str(manualSeed)+'.pth'
torch.save(model_ft.state_dict(), PATH)
model_ft.load_state_dict(torch.load(PATH))

<All keys matched successfully>